In [1]:
## Instalaciones

%pip install torch
%pip install open3d


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Dependencias

import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointnetClassifier, PointNetLoss
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cpu.


/home/antonioskorin/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# parametros de ejecución
checkpoint_freq = 10        # cada cuantos epoch guardar el modelo
ROOT_DIR = os.getcwd()
CHECKPOINT_DIR = os.path.join(ROOT_DIR, "checkpoint")

# parametros del dataset
classes = [ModelNetClass.MONITOR, ModelNetClass.TABLE, ModelNetClass.TOILET]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparametros
num_global_feats = 1024     # número de features globales calculadas
epochs = 10
learning_rate = 0.001
reg_weight = 0.001


train_data = ModelNet(classes, DatasetType.TRAIN)
validation_data = ModelNet(classes, DatasetType.VALIDATION)
test_data = ModelNet(classes, DatasetType.TEST)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
criterion = PointNetLoss(alpha=None, gamma=1, reg_weight=reg_weight, size_average=True).to(DEVICE)

In [4]:
loss_dict = {
    "train": {
        "loss": list(),
        "acc": list()
    },
    "valid": {
        "loss": list(),
        "acc": list()
    }
}

for epoch in range(1, epochs + 1):
    print(f"Epoch {epoch}:")
    '''
    Entrenamiento
    '''
    classifier = classifier.train()

    train_batch_losses = list()
    train_batch_accs = list()
    
    for pcds, labels in train_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)

        # Gradientes en cero
        optimizer.zero_grad()
        
        # Hacer predicciones
        out, _, A = classifier(pcds)
        loss = criterion(out, labels, A)
                
        # Calcular gradiente y optimizar
        loss.backward()
        optimizer.step()

        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas
        correct = pred_choice.eq(labels.data).cpu().sum()
        accuracy = correct.item() / float(pcds.size(0))

        # Registramos métricas para el batch
        train_batch_losses.append(loss.item())
        train_batch_accs.append(accuracy)
        
    epoch_train_loss = np.mean(train_batch_losses)
    epoch_train_acc = np.mean(train_batch_accs)
    print("  Train Loss:\t\t", epoch_train_loss)
    print("  Train Acc:\t\t", epoch_train_acc)
    loss_dict["train"]["loss"].append(epoch_train_loss)
    loss_dict["train"]["acc"].append(epoch_train_acc)

    '''
    Validación
    '''
    with torch.no_grad():
        classifier = classifier.eval()

        valid_batch_losses = list()
        valid_batch_accs = list()
        
        for pcds, labels in validation_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)
            
            # Hacer predicciones
            out, _, A = classifier(pcds)
            loss = criterion(out, labels, A)

            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas
            correct = pred_choice.eq(labels.data).cpu().sum()
            accuracy = correct.item() / float(pcds.size(0))

            # Registramos métricas para el batch
            valid_batch_losses.append(loss.item())
            valid_batch_accs.append(accuracy)
            
    epoch_valid_loss = np.mean(valid_batch_losses)
    epoch_valid_acc = np.mean(valid_batch_accs)
    print("  Validation Loss:\t", epoch_valid_loss)
    print("  Validation Acc:\t", epoch_valid_acc)
    loss_dict["valid"]["loss"].append(epoch_valid_loss)
    loss_dict["valid"]["acc"].append(epoch_valid_acc)

    if epoch % checkpoint_freq == 0:
        path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(epoch).zfill(4)}.pth")
        torch.save(classifier.state_dict(), path)

save_loss_dict(loss_dict)

Epoch 1:
  Train Loss:		 0.8112415671348572
  Train Acc:		 0.5303792569659443
  Validation Loss:	 134.5270334482193
  Validation Acc:	 0.328125
Epoch 2:
  Train Loss:		 0.6039788503395883
  Train Acc:		 0.6435758513931888
  Validation Loss:	 43.239521622657776
  Validation Acc:	 0.36328125
Epoch 3:
  Train Loss:		 0.4851116024349865
  Train Acc:		 0.6722136222910217
  Validation Loss:	 195.7159652709961
  Validation Acc:	 0.18802083333333333
Epoch 4:
  Train Loss:		 0.3117196999098125
  Train Acc:		 0.7669311145510836
  Validation Loss:	 3531.0287220925093
  Validation Acc:	 0.3203125
Epoch 5:
  Train Loss:		 0.34463616186066676
  Train Acc:		 0.760545665634675
  Validation Loss:	 133.49171674251556
  Validation Acc:	 0.359375
Epoch 6:
  Train Loss:		 0.21926452533194893
  Train Acc:		 0.8112422600619196
  Validation Loss:	 15.642842039465904
  Validation Acc:	 0.28567708333333336
Epoch 7:
  Train Loss:		 0.18887197971343994
  Train Acc:		 0.8336397058823529
  Validation Loss:	 30.4432